<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [2]:
lista_df = []

for archivo in archivos_anio:

    df_temp = pd.read_csv(archivo)
    df_temp.columns = df_temp.columns.str.lower()  

    lista_df.append(df_temp)

df_anio = pd.concat(lista_df, ignore_index=True)
df_anio

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.50,59.0
1,AGO,2001,30.20,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN
...,...,...,...,...
3055,WSM,2019,18.25,22.0
3056,YEM,2019,61.66,168.0
3057,ZAF,2019,22.19,31.0
3058,ZMB,2019,36.38,119.0


In [4]:
agrupado = df_codigos.groupby("codigo_iso")["pais"]
agrupado = agrupado.nunique().sort_values(ascending=False)

print("Codigo iso repetido : ", agrupado.index[0])

lista_paises = df_codigos[df_codigos["codigo_iso"] == agrupado.index[0]]["pais"]
lista_paises = lista_paises.tolist()

print("Paises asociados : ", lista_paises)

Codigo iso repetido :  ZWE
Paises asociados :  ['Zimbabue', 'malo']


In [5]:
# asumiendo que "MALO" es el error

indice = df_codigos.loc[df_codigos["pais"] == lista_paises[1]].index

df_codigos = df_codigos.drop(indice)

In [6]:
df = pd.merge(df_anio, df_codigos, on="codigo_iso", how="inner")
df

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.50,59.0,Afghanistán
1,AGO,2001,30.20,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos
...,...,...,...,...,...
3055,WSM,2019,18.25,22.0,Samoa
3056,YEM,2019,61.66,168.0,Yemen
3057,ZAF,2019,22.19,31.0,Sudáfrica
3058,ZMB,2019,36.38,119.0,Zambia




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [7]:
print("El conjunto de datos contiene ", df.shape[0], "Filas y ", df.shape[1], "Columnas")

print("Nombres de columnas: ", list(df.columns))

print("\nTipos de datos por columnas: \n", df.dtypes)

El conjunto de datos contiene  3060 Filas y  5 Columnas
Nombres de columnas:  ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos por columnas: 
 codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object


In [9]:
print(df.describe())

              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000


In [8]:
print("Valor maximo :", df["indice"].max(), "\nValor minimo:", df["indice"].min(), "\nvalor promedio: ", df["indice"].mean())

Valor maximo : 64536.0 
Valor minimo: 0.0 
valor promedio:  205.7823160660661


In [10]:
indice_min = df.loc[df["indice"].idxmin()]
indice_max = df.loc[df["indice"].idxmax()]
ranking_min = df.loc[df["ranking"].idxmin()]
ranking_max = df.loc[df["ranking"].idxmax()]

print("Mínimo índice:", indice_min["indice"], indice_min["pais"] )
print("Máximo índice: ", indice_max["indice"], indice_max["pais"] )
print("Mínimo ranking: ", ranking_min["indice"], ranking_min["pais"])
print("Máximo ranking: ", ranking_max["indice"], ranking_max["pais"])


Mínimo índice: 0.0 Dinamarca
Máximo índice:  64536.0 Kosovo
Mínimo ranking:  0.5 Finlandia
Máximo ranking:  64527.0 Kosovo


In [28]:
nulos_columna = df.isnull().sum()
print("Cantidad valores nulos :\n", nulos_columna)

nulos_porcentaje = nulos_columna/df.shape[0] *100
print("\nPorcentaje valores nulos :\n", nulos_porcentaje)

print("\nNo hay columnas con mas de 30% de datos faltantes")

Cantidad valores nulos :
 codigo_iso      0
anio            0
indice        396
ranking       223
pais            0
dtype: int64

Porcentaje valores nulos :
 codigo_iso     0.000000
anio           0.000000
indice        12.941176
ranking        7.287582
pais           0.000000
dtype: float64

No hay columnas con mas de 30% de datos faltantes


In [32]:
print("En el dataframe hay ", df['pais'].nunique(), "Paises distintos y ", df['anio'].nunique(), "años distintos" )

print("Filas duplicadas: ", df.duplicated().sum())

En el dataframe hay  179 Paises distintos y  17 años distintos
Filas duplicadas:  0


In [33]:
inconsistencias = df.groupby("codigo_iso")["pais"].nunique()
inconsistencias = inconsistencias[inconsistencias > 1]

print(inconsistencias) 

print("\nNo hay inconsistencias entre codigo y pais")

Series([], Name: pais, dtype: int64)

No hay inconsistencias entre codigo y pais





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [14]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america =  df.loc[df["codigo_iso"].isin(america)]
df_america 

,codigo_iso,anio,indice,ranking,pais
5,ARG,2001,12.00,8.0,Argentina
7,ATG,2001,NaN,NaN,Antigua y Barbuda
20,BLZ,2001,NaN,NaN,Belize
21,BOL,2001,14.50,13.0,Bolivia
22,BRA,2001,18.80,18.0,Brasil
...,...,...,...,...,...
3029,SUR,2019,16.38,20.0,Surinam
3043,TTO,2019,24.74,39.0,Trinidad y Tobago
3050,URY,2019,16.06,19.0,Uruguay
3051,USA,2019,25.69,48.0,Estados Unidos


In [15]:
lista_años = df_america["anio"].unique()

for año in lista_años:

    df_temp = df_america[df_america["anio"] == año]
    valores = df_temp["indice"].dropna()

    if len(valores) > 0:
        #print(año)
        indice = df_temp["indice"].idxmin()
        min_temp = df_temp.loc[indice]["pais"]

        indice = df_temp["indice"].idxmax()
        max_temp = df_temp.loc[indice]["pais"]

        print("\nEn el año ", año, "\nel mejor pais fue :", min_temp, "\nEl peor pais fue :", max_temp)

    else:
        print("\nEl año ", año, "no tiene datos válidos")


En el año  2001 
el mejor pais fue : Canadá 
El peor pais fue : Cuba

En el año  2002 
el mejor pais fue : Trinidad y Tobago 
El peor pais fue : Cuba

En el año  2003 
el mejor pais fue : Trinidad y Tobago 
El peor pais fue : Argentina

En el año  2004 
el mejor pais fue : Trinidad y Tobago 
El peor pais fue : Cuba

En el año  2005 
el mejor pais fue : Bolivia 
El peor pais fue : Cuba

En el año  2006 
el mejor pais fue : Canadá 
El peor pais fue : Cuba

En el año  2007 
el mejor pais fue : Canadá 
El peor pais fue : Cuba

En el año  2008 
el mejor pais fue : Canadá 
El peor pais fue : Cuba

En el año  2009 
el mejor pais fue : Estados Unidos 
El peor pais fue : Cuba

En el año  2012 
el mejor pais fue : Jamaica 
El peor pais fue : Cuba

En el año  2013 
el mejor pais fue : Jamaica 
El peor pais fue : Cuba

En el año  2014 
el mejor pais fue : Canadá 
El peor pais fue : Cuba

En el año  2015 
el mejor pais fue : Costa Rica 
El peor pais fue : Cuba

El año  2016 no tiene datos válidos


In [16]:
df_america_filtrado = df_america.dropna(subset=["indice"])

mejores = df_america.loc[df_america_filtrado.groupby("anio")["indice"].idxmin()]
mejores = mejores.drop(["codigo_iso", "ranking"], axis=1)
mejores.columns = ["Año", "Mejor indice", "Mejor Pais"]

peores = df_america.loc[df_america_filtrado.groupby("anio")["indice"].idxmax()]
peores = peores.drop(["codigo_iso", "ranking"], axis=1)
peores.columns = ["Año", "Peor indice", "Peor Pais"]

print(pd.merge(mejores, peores, on="Año", how="outer"))

     Año  Mejor indice         Mejor Pais  Peor indice  Peor Pais
0   2001          0.80             Canadá        90.30       Cuba
1   2002          1.00  Trinidad y Tobago        97.83       Cuba
2   2003          2.00  Trinidad y Tobago     35826.00  Argentina
3   2004          2.00  Trinidad y Tobago        87.00       Cuba
4   2005          4.50            Bolivia        95.00       Cuba
5   2006          4.88             Canadá        96.17       Cuba
6   2007          3.33             Canadá        88.33       Cuba
7   2008          3.70             Canadá        94.00       Cuba
8   2009          6.75     Estados Unidos        78.00       Cuba
9   2012          9.88            Jamaica        71.64       Cuba
10  2013         10.90            Jamaica        70.92       Cuba
11  2014         10.99             Canadá        70.21       Cuba
12  2015         11.10         Costa Rica        70.23       Cuba
13  2017         11.93         Costa Rica        71.75       Cuba
14  2018  

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [34]:
pivot_table = pd.pivot_table(df, index="pais", columns="anio", values="indice", aggfunc="max", fill_value=0)

pivot_table = pivot_table.replace(0, np.nan)

pivot_table


anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,NaN,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


In [ ]:
max_indice = pivot_table.max().max()
max_pais = pivot_table.max(axis = 1).idxmax()

min_indice = pivot_table.loc[(pivot_table > 0).all(axis=1)].min().min()
min_indice = min_indice.min().min()
min_pais = pivot_table.min(axis = 1).idxmin()

#min_indice

print("El pais con mayor valor es", max_pais, "con", max_indice )
print("\nEl pais con menor valor es", min_pais, "con", min_indice )

El pais con mayor valor es Kosovo con 64536.0

El pais con menor valor es Austria con 0.5


In [36]:
promedios_año = pivot_table.mean(axis=0)

año_max_prom = promedios_año.idxmax()
año_min_prom = promedios_año.idxmin()

print("El año con mayor promedio es",  año_max_prom, "con :", promedios_año.max(), "\nEl año con menor promedio ", año_min_prom, "con :", promedios_año.min())

El año con mayor promedio es 2012 con : 468.88350877192977 
El año con menor promedio  2002 con : 25.879683544303795


In [37]:
variabilidad = pivot_table.max(axis=1) - pivot_table.min(axis=1)
pais_max_var = variabilidad.idxmax()

print("El pais con mayor variabilidad es :", pais_max_var,  "con :", variabilidad.max())

El pais con mayor variabilidad es : Kosovo con : 46098.0


In [38]:
constantes = pivot_table.nunique(axis=1)
constantes = constantes[constantes == 1]

print("Existen", len(constantes), "paises con indice constante:\n", constantes.index.tolist())

Existen 2 paises con indice constante:
 ['Antigua y Barbuda', 'Granada']


In [39]:
nodatos = (pivot_table == 0)
nodatos = nodatos.all(axis=1)
nodatos = pivot_table[nodatos].index.tolist()

print("los paises sin datos :", nodatos)

los paises sin datos : []
